# 02 - The Fourier Motzkin Elimination Algorithm

**Goal:** Project a polyhedron by eliminating one variable.

We start with a polyhedron $P(A,b)=\{x\in K^m \mid Ax \le b\}$, where $K \subseteq R$.

Eliminating variable $x_k$ gives a new inequality system in the remaining variables that describes the projection of $P$ onto the coordinates without $x_k$:

${Proj}_k(P(A,b))=\{x\in K^m \mid x_k=0, \exists \lambda \in K : x + \lambda e^k \in P(A,b)\}$

This notebook will:
1. Give a small 2D example (so the projection becomes easy to understand)
2. Show the elimination rule (combining "+" and "-" constraints)
3. Use a reusable implementation from `src/`

**Remove?**

This algorithm computes the orthogonal projection of a polyhedron by eliminating a given axis. <br><br>
Assume the original polyhedron is defined by $P(A,b)=\{x\in K^m:Ax\leq b\}$, where $K \subseteq R$. <br>
Its orthogonal projection is given by ${Proj}_k(P(A,b))=\{x\in K^m : x_k=0, \exists \lambda \in K : x + \lambda e^k \in P(A,b)\}$, which is the projection of the polyhedron's shadow along dimension $k$.<br><br>
This projection can be obtained using the Fourier Motzkin Elimination (FME) Algorithm, and it can be shown that its result is again a polyhedron given by ${Proj}_k(P(A,b))=P(D,d)=\{x\in K^m:Dx\leq d, x_k=0\}$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from src import visualize_polyhedron

# A tiny 2D polyhedron in (x, y):
# x >= 0, y >= 0, x <= 3, y <= 3, x + y <= 4
A = np.array(
    [
        [-1.0,  0.0],  # -x <= 0  -> x >= 0
        [ 0.0, -1.0],  # -y <= 0  -> y >= 0
        [ 1.0,  0.0],  #  x <= 3
        [ 0.0,  1.0],  #  y <= 3
        [ 1.0,  1.0],  #  x + y <= 4
    ]
)
b = np.array([0.0, 0.0, 3.0, 3.0, 4.0]).reshape(-1, 1)

visualize_polyhedron(A_mat=A, b_mat=b)

## Step 1 - Split inequalities by the sign of the eliminated coefficient

To eliminate $x_k$, we partition the inequalities $a_i^\top x \le b_i$ into three index sets:

- $I_0 = \{ i \mid a_{ik} = 0\}$ (Constraints that do not involve $x_k$; they remain unchanged)
- $I_+ = \{ i \mid a_{ik} > 0\}$ (These give **upper bounds** on $x_k$)
- $I_- = \{ i \mid a_{ik} < 0\}$ (Giving **lower bounds** on $x_k$)

Fourier-Motzkin elimination (FME) will keep all constraints in $I_0$, and generate new constraints by combining each $i \in I_-$ with each $j \in I_+$.

In [ ]:
k = 1  # --> eliminate axis 1 (y)
ak = A[:, k]

I0 = np.where(np.isclose(ak, 0.0))[0]
Ip = np.where(ak > 0.0)[0]
In = np.where(ak < 0.0)[0]

print("I0 (ak = 0):", I0)
print("I+ (ak > 0):", Ip)
print("I- (ak < 0):", In)
print("Expected #new constraints =", len(I0) + len(Ip) * len(In))

## Step 2 - Combine a lower and an upper bound to eliminate $x_k$

Take one inequality from $I_+$: $a_j^\top x \le b_j,\quad a_{jk} > 0$. This can be rearranged into an **upper bound** on $x_k$.

Take one inequality from $I_-$: $a_i^\top x \le b_i,\quad a_{ik} < 0$. This gives a **lower bound** on $x_k$.

FME eliminates $x_k$ by enforcing $\text{(lower bound)} \le \text{(upper bound)}$ and multiplying to avoid division. \
A numerically stable way (and what we implement) is the linear combination:

$a_{jk}(a_i^\top x) - a_{ik}(a_j^\top x) \le a_{jk} b_i - a_{ik} b_j$.

This new inequality has **zero coefficient** in position $k$, so it lives in the projected space.

**Note:** This can cause a **combinatorial blow-up**, as the number of inequalities can grow like $|I_-|\cdot |I_+|$ in each elimination step. \
FME is conceptually simple and very useful for small dimensions, projections, and proofs—but for larger problems it needs pruning/redundancy removal to stay practical.

## Implementation in `src/`

Below is the implementation used in this tutorial (pulled from `src/polyhedra/fourier_motzkin.py` during the book build).

### Fourier-Motzkin Elimination

```{literalinclude} ../src/polyhedra/fourier_motzkin.py
:language: python
:linenos:
:start-after: "# --- FME:eliminate:start"
:end-before: "# --- FME:eliminate:end"
```

### Interpreting a 1D system as an interval

```{literalinclude} ../src/polyhedra/fourier_motzkin.py
:language: python
:linenos:
:start-after: "# --- FME:bounds:start"
:end-before: "# --- FME:bounds:end"
```

Let's run it! 🙂

In [ ]:
from src.polyhedra.fourier_motzkin import fourier_motzkin_eliminate, constraints_1d_bounds

# Eliminate y (index 1) -> projection onto x-axis
A_proj, b_proj, stats = fourier_motzkin_eliminate(A, b, k=1, drop_k=True)

print(stats)
print("Projected system (in x only):")
print(np.concatenate([b_proj, A_proj], axis=1))

L, U = constraints_1d_bounds(A_proj[:, 0], b_proj[:, 0])
print(f"Projection interval for x: [{L:.3f}, {U:.3f}]")

In [ ]:
xs = np.linspace(-1, 5, 200)
mask = (xs >= L) & (xs <= U)

plt.figure()
plt.plot(xs, mask.astype(int))
plt.yticks([0, 1], ["infeasible", "feasible"])
plt.xlabel("x")
plt.title("Feasible x after eliminating y (projection)")
plt.grid(True)
plt.show()

## Takeaways

- **Fourier–Motzkin elimination (FME)** projects a polyhedron $Ax \le b$ by *eliminating variables* and producing a new inequality system in fewer variables.
- The method works by splitting inequalities into three groups w.r.t. the eliminated variable $x_k$: $I_0$, $I_-$, and $I_+$.
- New constraints are then generated by pairing each $i \in I_-$ with each $j \in I_+$.

## Next

Next up: **Affine mappings and polar cones**.

➡️ [03 - Affine Mappings and Polar Cones](03-Affine-Mappings-and-Polar-Cones.ipynb)

You’ll see how polyhedra transform under linear/affine maps and how polarity gives a powerful dual geometric view. \
Both ideas are repeatedly useful when reasoning about projections, representations, and (later) cutting planes.